# 실험 09-01 증가형 MOSFET의 전압-전류 특성

## 공통 코드

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
from scipy.signal import savgol_filter # 사비츠키-골레이(Savitzky-Golay) 필터
from numpy import nan, inf

# %matplotlib tk
# %matplotlib inline

def tangent_line(f, x):
    h = 1e-4
    d = (f(x+h) - f(x-h)) / (2*h)
    return lambda t: d*t - d*x + f(x)

def derivative(f, x):
    h = 1e-4
    d = (f(x+h) - f(x-h)) / (2*h)
    return d

def get_simulation_result(file_name, start=0, end=-1):
    if end == -1:
        end = None
    else:
        end += 1
    
    with open(file_name, encoding='cp1252') as data_file:
        lines = data_file.read()
        occurrences = lines.count('Step Information:')
        data_file.seek(0)

        line = data_file.readline()
        labels = re.split(', | ,|\t', line)
        labels = [s.strip().upper() for s in labels]

        data = {}

        if (occurrences == 0):
            for label in labels:
                data[label] = []
            
            for line in data_file:
                values = re.split(', | ,|\t', line)
                for i in range(len(values)):
                    value = float(values[i]) * 1000
                    data[labels[i]].append(value)

        else:
            labels_all =[]
            
            lines = data_file.readline() # skip first line starting with 'Step Information:'

            for idx in range(occurrences):
                labels_new = []
                for label in labels:
                    labels_new.append('(%s)@%d' % (label, idx+1))
                labels_all += labels_new

                for label_new in labels_new:
                    data[label_new] = []

                for line in data_file:
                    if (line.startswith('Step Information:')):
                        break
                    values = re.split(', | ,|\t', line)
                    for i in range(len(values)):
                        value = float(values[i]) * 1000
                        data[labels_new[i]].append(value)

            labels = labels_all

        for label in labels:
            data[label] = np.array(data[label][start:end])

    # print("labels = ", end='')
    # print(list(data.keys()))
    for label in list(data.keys()):
        print("data['%s'] : sample number = %d" % (label, len(data[label])))

    return data

def get_oscilloscpoe_result_tektronix(file_name, start=0, end=-1):
    if end == -1:
        end = None
    else:
        end += 1

    df = pd.read_csv(file_name, header=None, encoding='cp1252',low_memory=False)

    label_ri = df.loc[df.iloc[:,0] == 'Source', 1].index
    ci = 0
    data = {}
    data['TIME'] = df.iloc[start:end, 3].to_numpy().astype(float)
    data['TIME'] -= data['TIME'][0]
    for idx in range(df.shape[1]//6):
        label = df.iloc[label_ri, ci+1].item()
        data[label] = df.iloc[start:end,ci+4].to_numpy().astype(float)
        ci += 6

    # print("labels = ", end='')
    # print(list(data.keys()))
    for label in list(data.keys()):
        print("data['%s'] : sample number = %d" % (label, len(data[label])))

    return data

def print_array(label, values):
    print('%s = [' % label, end='')
    for idx, vd in enumerate(values):
        print('{:11.3f}'.format(vd), end='')
        if (idx+1 != len(values)):
            print(', ', end='')
    print(']')

def print_value(label, value):
    print('%s = ' % label, end='')
    print('{:11.3f}'.format(value))

def draw_plot(xs, ys, label, style_idx, marker_num=16, scatter=False, scatter_s=2):
    linestyle  = ['-',          '-',            '-',            '-',            '-', 
                  '-',          '-',            '-',            '-',            '-', 
                  '-',          '-',            '-',            '-',            '-', 
                  '-',          '-',            '-',            '-',            '-', 
                  '-',          '-',            '-',            '-',            '-']
    colors     = ['blue',       'green',        'red',          'orange',       'purple',
                  'cyan',       'darkseagreen', 'brown',        'goldenrod',    'darkviolet',
                  'steelblue',  'limegreen',    'tomato',       'tan',          'deeppink',
                  'navy',       'lightgreen',   'indianred',    'khaki',        'rebeccapurple',
                  'slategray',  'forestgreen',  'orangered',    'wheat',        'orchid']
    markers    = ['o',          'v',            '<',            's',            'p',
                  'h',          '*',            'X',            'x',            '^',
                  '>',          'P',            'D',            'H',            'd',
                  '|',          '-',            '4'             '5',            '6',
                  '7',          '8',            '9 ',           '10',           '+']

    if isinstance(xs, list):
        xs = np.array(xs)
    if isinstance(ys, list):
        ys = np.array(ys)

    if (marker_num == 0):
        selected_markevery = None
        selected_marker = None
    elif not isinstance(xs, np.ndarray):
        selected_markevery = None
        selected_marker = markers[style_idx]
    elif (marker_num > 0):
        markeverys = []
        if xs[0] > xs[-1]:
            first_idx = -1
        else:
            first_idx = 0
        x_step = (xs.max() - xs.min()) / marker_num
        for idx in range(len(markers)):
            x_start = xs.min() + ((x_step / (len(markers) + 1)) * idx)
            marker_list = []
            for j in range(marker_num):
                tmp_array = np.where(xs > (x_start + (x_step * j)))
                if (len(tmp_array[0]) > 0):
                    marker_list.append(tmp_array[0][first_idx])
            markeverys.append(marker_list)

        selected_markevery = markeverys[style_idx]
        selected_marker = markers[style_idx]
    else:
        selected_markevery = None
        selected_marker = markers[style_idx]

    if (scatter):
        plt.scatter(xs, ys, color=colors[style_idx], marker=selected_marker, s=scatter_s, label=label)
    else:
        plt.plot(xs, ys, ls=linestyle[style_idx], color=colors[style_idx], marker=selected_marker, markevery=selected_markevery, label=label)
    if (label != None):
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)

def set_plot_size(xrate, yrate):
    if (len(set_plot_size.g_plt_figsize) == 0):
        set_plot_size.g_plt_figsize = plt.rcParams["figure.figsize"]
    size = []
    size.append(set_plot_size.g_plt_figsize[0] * xrate)
    size.append(set_plot_size.g_plt_figsize[1] * yrate)
    plt.figure(figsize=size)

set_plot_size.g_plt_figsize = []

## LTspice 명령

* 컴포넌트 추가 명령
    + Voltage source: Component -> voltage (V)
    + Resister: Component -> res (R)
    + Diode: diode (D)
    + Zener diode: Component -> zenor
    + Bipolar NPN transistor: Component -> npn
    + Bipolar PNP transistor: Component -> pnp
    + N-Channel MOSFET transistor: Component -> nmos
    + Ground: ground (G)

&nbsp;

* SPICE 시뮬레이션 설정 명령
    * DC Sweep analysis
        + Compute the DC operating point over swept independent source(s).
        + Syntax: .dc \<source1\> \<start1\> \<stop1\> [\<increment1\>] [\<source2\> ...]
            + source: source name
            + start: start value
            + stop: stop value
            + increment: increment value
    * Transient analysis
        + Performa time-domain analysis
        + Syntax: .tran \<Tprint\> \<Tstop\> [\<Tstart\> [\<Tmaxstep\>]]
            - Tprint: printing increment (Use 0 in LTspice)
            - Tstop: stop time
            - Tstart: time to start saving waveform data
            - Tmaxstep: maximum timestep size
    * Parametric analysis (list)
        + Performs a parametric sweep for all of the analysis
        + Syntax: .step param \<name\> list \<value1\> [\<value2\> [\<value3\> [...]]]
            - name: name of parameter to step
    + DC Bias Point  analysis
        + Performa DC bias point analysis
        + Syntax: .op

&nbsp;

* 기타 명령
    + 회전 (Control + R)
    + Wire (F3)
    + Move (F7)
    + Drag (F8)
    + Delete (F5)
    + Duplicate (F6)


*******************************************************************************
## 시뮬레이션 1-1 결과 분석

## 회로도

![Markdown-mark](../ltspice/simulation_01_01.png)

### 1. 시뮬레이션 결과 파일을 읽는다.

In [ ]:
data_l = get_simulation_result('../ltspice/simulation_01_01.txt')

### 2. 시뮬레이션 결과에서 $V_{DS}$, $I_{D}$를 가져온다.

In [ ]:
V_GG_V_a = np.array([1.0, 2.0, 2.5, 3.0, 3.5])

V_DS_V_a_l = []
I_D_mA_a_l = []

for idx in range(len(V_GG_V_a)):
    V_DS_V_a_l.append(data_l['(V(VDS))@%d' % (idx+1)] / 1000.)
    I_D_mA_a_l.append(data_l['(ID(M1))@%d' % (idx+1)])
#    I_D_mA_a_l.append(data_l['(IX(U1:D))@%d' % (idx+1)])

V_DS_V_a = np.array([0.1, 0.4, 0.8, 2.0, 4.0, 5.0])
print_array("              V_DS_V", V_DS_V_a)
for idx in range(len(V_GG_V_a)):
    I_D_mA_a = []
    for V_DS_V in V_DS_V_a:
        I_D_mA_a.append(np.interp(V_DS_V, V_DS_V_a_l[idx], I_D_mA_a_l[idx]))
    label = 'V_GG_V = {:3.1f}: I_D_mA'.format(V_GG_V_a[idx])
    print_array(label, I_D_mA_a)

### 3. $I_{D}$ - $V_{DS}$ 그래프를 그린다.

In [ ]:
plt.xlabel('V_DS_V')
plt.ylabel('I_D_mA')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
for idx in range(len(V_GG_V_a)):
    draw_plot(V_DS_V_a_l[idx], I_D_mA_a_l[idx], '(V_GG_V={:3.1f})'.format(V_GG_V_a[idx]), idx)
plt.show()

In [ ]:
plt.xlabel('V_DS_V')
plt.ylabel('I_D_mA')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
for idx in range(len(V_GG_V_a)):
    draw_plot(V_DS_V_a_l[idx], I_D_mA_a_l[idx], '(V_GG_V={:3.1f})'.format(V_GG_V_a[idx]), idx, marker_num=5)

V_DD_V = 15.
R_D_kOhm_a = [0.075, 0.10, 0.15, 0.20, 0.27, 0.43]

for idx in range(len(R_D_kOhm_a)):
    draw_plot([0, V_DD_V], [V_DD_V/(R_D_kOhm_a[idx]), 0], '(R_D_Ohm={:3.2f})'.format(R_D_kOhm_a[idx]), idx, marker_num = 0)

plt.show()

*******************************************************************************
## 시뮬레이션 2-1 결과 분석

## 회로도

![Markdown-mark](../ltspice/simulation_02_01.png)

### 1. 시뮬레이션 결과 파일을 읽는다.

In [ ]:
data_l = get_simulation_result('../ltspice/simulation_02_01.txt')

### 2. 시뮬레이션 결과에서 $V_{GG}$, $I_{D}$를 가져온다.

In [ ]:
V_DS_V = 4.
V_GG_V_a2 = [   0.4,    1.0,    2.0,    2.5,    3.0,    3.5,    4.0,    4.5,    5.0]
I_D_mA_a2 = []
V_GG_V_a = data_l['V(VGG)'] / 1000.
#I_D_mA_a = data_l['IX(U1:D)']
I_D_mA_a = data_l['ID(M1)']
for V_GG_V in V_GG_V_a2:
    I_D_mA_a2.append(np.interp(V_GG_V, V_GG_V_a, I_D_mA_a))

print_array("V_GG_V", V_GG_V_a2)
print_array('I_D_mA', I_D_mA_a2)

### 3. $I_{D}$ - $V_{GS}$ 그래프를 그린다.

In [ ]:
plt.xlabel('V_GS_V')
plt.ylabel('I_D_mA')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(V_GG_V_a, I_D_mA_a, None, 0)
plt.show()

### 3. 문턱 전압 $V_{Tn}$을 구한다. (Second-Derivative (SD) method 사용)

In [ ]:
fx = lambda x: np.interp(x, V_GG_V_a, I_D_mA_a)

x = np.arange(V_GG_V_a[0], V_GG_V_a[-1], 0.01)
y = fx(x)
y = savgol_filter(y, 100, 2)

g = derivative(fx, x)
g = savgol_filter(g, 100, 2)

f2x = lambda x2: np.interp(x2, x, g)
g2 = derivative(f2x, x)
g2 = savgol_filter(g2, 100, 2)

g2_max = g2.max()
x_g2_max = x[g2.argmax()]

V_Tn_V = x_g2_max

plt.xlabel('V_GS_V')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(x, y, 'I_D_mA (V_DS_V=%4.3f)' % V_DS_V, 0, marker_num=0)
draw_plot(x, g, 'gm', 1, marker_num=0)
draw_plot(x, g2, 'gm2', 2, marker_num=0)
plt.scatter(x_g2_max, g2_max, color='r', zorder=10)
plt.show()


print_value("V_Tn_V", V_Tn_V)

### 4. 문턱 전압 $V_{Tn}$을 구한다. (Linear Extrapolation (LE) method 사용)

In [ ]:
fx = lambda x: np.interp(x, V_GG_V_a, I_D_mA_a)

x = np.arange(V_GG_V_a[0], V_GG_V_a[-1], 0.01)
y = fx(x)

g = derivative(fx, x)
g_max = g.max()
x_g_max = x[g.argmax()]

tf = tangent_line(fx, x_g_max)
tx = np.arange(2.8, 5.0, 0.01)
ty = tf(tx)
V_Tn_V = np.interp(0.0, ty, tx)

plt.xlabel('V_GS_V')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(x, y, 'I_D_mA (V_DS_V=%4.3f)' % V_DS_V, 0)
draw_plot(x, g, 'gm', 1)
plt.scatter(x_g_max, g_max, color='r', zorder=10)
plt.axvline(x_g_max, color='r')
plt.plot(tx, ty, 'r-')
plt.scatter(V_Tn_V, 0, color='r', zorder=10)
plt.text(V_Tn_V, -30, '%4.3f' % V_Tn_V)
plt.show()

print_value("V_Tn_V", V_Tn_V)